In [1]:
import os
# Set spark environments
os.environ['PYSPARK_PYTHON'] = r'C:\Users\room102sys2\AppData\Local\Programs\Python\Python39'
#os.environ['PYSPARK_DRIVER_PYTHON'] = r'C:\Users\room102sys2\AppData\Local\Programs\Python\Python39\Scripts\jupyter'
#os.environ['']

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [3]:
import findspark
findspark.init('C:\Program Files\Spark')

In [4]:
sp = SparkSession.builder.appName("SparkQL").getOrCreate()
sc = sp.sparkContext

# RESILENT DISTRIBUTED DATASETS

There are two sets of low-level APIs: 
- there is one for manipulating distributed data (RDDs), and
- another for distributing and manipulating distributed shared variables (broadcast variables and accumulators).

**When to Use the Low-Level APIs?**
- When we need some functionality that we cannot find in higer level APIs
- When we need to maintain legacy codebase written using RDDs
- When we need to do some custom shared variable manipulation 

**How to Use the Low-Level APIs?**
- A `SparkContext` is the entry point for low-level API functionality.
- We can access it through the `SparkSession`, which is the tool we use to perform computation across a Spark cluster.

In [ ]:
sc = sp.sparkContext

## About RDDs

- RDD represents an immutable, partitioned collection of records that can be operated on in parallel, unlike `DataFrame` through where each record is a structured row containing fields with known schema, in RDDs the records are just JAVA, Scala, or Python objects of the programmer's choosing.
- RDDs give us complete control because every record in an RDD is a just a Java or Python object.

## Types of RDDs
As a user, we will likely only be creating two types of RDDs:
- The generic 
- key-value i.e, that provieds additional functions, such as aggregating by key.

**each RDD is characterized by five main properties:**
- A list of partitions
- A function for computing each split
- A list of dependencies on other RDDs
- Optionally, a list of preferred locations on which to compute each split(e.g., block locations for a Hadoop Distributed File System [HDFS] file)
- Optionally, a `partitioner` for key-value RDDs (e.g., to say that the RDD is hash-partitioned)

**For Scala and Java, the performance is for the most part the same, the large costs incurred in manipulating the raw objects but in python can lose substantial amount of performance when using RDDs.**

Running Python RDDs equates to running Python user-defined functions (UDFs) row by row.

### When to use RDDs ?
The most likely reason for use RDDs is because we need fine-grained control over the physical distribution of data (custom partitioning of data)

**what is the difference between RDDs of Case Classes and Datasets?**

The difference is that Datasets can still take advantage of the wealth of functions and optimizations that the Structured APIs have to offer. With Datasets, we do not need to choose between only operating on JVM types or on Spark types, we can choose
whatever is either easiest to do or most flexible.

## Creating RDDs

### Interoperating Between DataFrames, Datasets, and RDDs

One of the easiest ways to get RDDs is from an existing DataFrame or Dataset. Converting these to an
RDD is simple: just use the rdd method on any of these data types.

In [5]:
# in Python
sp.range(10).rdd

MapPartitionsRDD[5] at javaToPython at <unknown>:0

To operate on this data, we need to convert this Row object to the correct data type or extract
values out of it.

In [6]:
# in Python
sp.range(10).toDF("id").rdd.map(lambda row: row[0])

PythonRDD[12] at RDD at PythonRDD.scala:53

We can use the same methodology to create a DataFrame or Dataset from an RDD. All we need to
do is call the toDF method on the RDD:

In [7]:
# in Python
sp.range(10).rdd.toDF()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (room102csesys9 executor driver): java.io.IOException: Cannot run program "C:\Users\room102sys2\AppData\Local\Programs\Python\Python39": CreateProcess error=5, Access is denied
	at java.lang.ProcessBuilder.start(Unknown Source)
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:165)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD

### From a Local Collection
- To create an RDD from a collection, you will need to use the `parallelize` method on a SparkContext (within a SparkSession).
- This turns a single node collection into a parallel collection.
- When creating this parallel collection, you can also explicitly state the number of partitions into which you would like to distribute this array. 

In [5]:
# in Python
myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple".split(" ")
words = sp.sparkContext.parallelize(myCollection, 2)

An additional feature is that we can then name this RDD to show up in the Spark UI according to a
given name:

In [10]:
# in Python
words.setName("myWords")
words.name() # myWords

'myWords'

### From Data Sources

- We can create RDDs from data sources or text files, it’s often preferable to use the Data Source APIs.
- RDDs do not have a notion of “Data Source APIs” like DataFrames do; they primarily define their dependency structures and lists of partitions.
- We can also read data as RDDs using Spark Context

In [ ]:
sp.sparkContext.textFile("\some\path\withTextFiles")

- Alternatively, we can read in data for which each text file should become a single record. 
- The use case here would be where each file is a file that consists of a large JSON object or some document that we will operate on as an individual

`spark.sparkContext.wholeTextFiles("/some/path/withTextFiles")`

In this RDD, the name of the file is the first object and the value of the text file is the second string
object.

### Manipulating RDDs

- We manipulate RDDs in much the same way that you manipulate DataFrames. The difference being that we manipulate raw Java or Scala objects instead of Spark types.
- There is also a dearth of “helper” methods or functions that you can draw upon to simplify calculations.

- We must define each filter, map functions, aggregation, and any other manipulation that we want as a function.


## Transformations

Just as we do with DataFrames and DataSets, we specify transformations on one RDD to create another. In doing so, we define an RDD as a dependency to another along with some manipulation of the data contained in that RDD.


### distinct
A `distinct` method call on an RDD removes duplicates from the RDD:

`words.distinct().count()`

That gives us a result of 10.

### filter

Filtering is equivalent to creating a SQL-like where clause.
We can look through our records in the RDD and see which ones match some predicate function. This function just needs to return a Boolean type to be used as filter function. The input should be whatever we given row is.

In [6]:
words.distinct().count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 0.0 failed 1 times, most recent failure: Lost task 1.0 in stage 0.0 (TID 1) (room102csesys9 executor driver): java.io.IOException: Cannot run program "C:\Users\room102sys2\AppData\Local\Programs\Python\Python39": CreateProcess error=5, Access is denied
	at java.lang.ProcessBuilder.start(Unknown Source)
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:165)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spar

In [8]:
# in Python
def startsWithS(individual):
    return individual.startswith("S")

Above code gives a result of *Spark* and *Simple* . We can see, like the Dataset API, that this returns native types. That is because we never coerce our data into type `Row`, nor do we need to convert the data after collecting it.

### map

- We specify a function that returns the value that we want, given the correct input.
- We then apply that, record by record.

In [9]:
# in Python
words2 = words.map(lambda word: (word, word[0], word.startswith("S")))

We can subsequently filter on this by selecting the relevant Boolean value in a new function:

In [10]:
# in Python
words2.filter(lambda record: record[2]).take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (room102csesys9 executor driver): java.io.IOException: Cannot run program "C:\Users\room102sys2\AppData\Local\Programs\Python\Python39": CreateProcess error=5, Access is denied
	at java.lang.ProcessBuilder.start(Unknown Source)
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:165)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD

This returns a tuple of “Spark,” “S,” and “true,” as well as “Simple,” “S,” and “True.”

#### flatMap
`flatMap` provides a simple extension of the map function we just looked at. Sometimes, each current row should return multiple rows, instead.
`flatMap` requires that the ouput of the map function be an iterable that can be expanded

In [11]:
# in Python
words.flatMap(lambda word: list(word)).take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3) (room102csesys9 executor driver): java.io.IOException: Cannot run program "C:\Users\room102sys2\AppData\Local\Programs\Python\Python39": CreateProcess error=5, Access is denied
	at java.lang.ProcessBuilder.start(Unknown Source)
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:165)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD

### sort
To sort an RDD we must use the `sortBy` method, and just like any other RDD operation, we do this by specifying a function to extract a value from the objects in your RDDs and then sort based on that. \ For Instance, the following example sorts by word length from longest to shortest.

In [ ]:
# in Python
words.sortBy(lambda word: len(word) * -1).take(2)

### Random Splits
We can also randomly split an RDD into an Array of RDDs by using the randomSplit method,
which accepts an `Array` of weights and a random seed:

In [12]:
# in Python
fiftyFiftySplit = words.randomSplit([0.5, 0.5])

Above code returns an array of RDDs that you can manipulate individually.

## Actions
Just as we do with DataFrames and Datasets, we specify actions to kick off our specified
transformations. Actions either collect data to the driver or write to an external data source.